In [1]:
import os
import sys
import carla
import yaml
import csv
import json
import pandas as pd
import numpy as np
import shutil
import glob
from PIL import Image, ImageSequence

project_directory = os.path.split(os.getcwd())[0] + '/'
print(project_directory)
sys.path.append(project_directory)

import utils_cogmod
from utils_cogmod.server_utils import CarlaServerManager

/home/ubuntu/vihaan-devel/carla/sip-report-gen/carla-roach-0.9.13/


In [2]:
#Get user input on which accident they would like to analyze

def user_input():
    global town
    global traffic_density
    global driver_condition
    global trial_num
    
    town = int(input("Town04 (4) or Town05 (5)?"))
    traffic_density = int(input("Low (1) or Medium (2) or High (3) traffic density?"))
    date = int(input("Which set of tests would you like to select, 00-06-31 (1), 00-14-25 (2), 03-03-02 (3), 23-40-09 (4)?"))
    driver_condition = int(input("Normal (1) or Distracted (2)?"))
    trial_num = input("0 or 1 or 2 or 3 or 4")

    if town == 4:
        town = "Town04"
    elif town == 5:
        town = "Town05"

    if traffic_density == 1:
        traffic_density = "low"
    elif traffic_density == 2:
        traffic_density = "medium"
    elif traffic_density == 3:
        traffic_density = "high"

    if date == 1:
        date = "00-06-31"
    elif date == 2:
        date = "00-14-25"
    elif date == 3:
        date = "03-03-02"
    elif date == 4:
        date = "23-40-09"

    if driver_condition == 1:
        driver_condition = "normal"
    elif driver_condition == 2:
        driver_condition = "distracted"

    folder = "CogMod-v0_" + town + "_" + traffic_density + "_simple"
    file_path = driver_condition + "_1_repeat_" + trial_num

    print("Town: {}, Traffic Density: {}, Date: {}, Driver Condition: {}, Trial Number: {}, Folder: {}, File Path: {}".format(town, traffic_density, date, driver_condition, trial_num, folder, file_path))
    
    return town, traffic_density, date, driver_condition, trial_num, folder, file_path

In [3]:
#Setting up config_path and Carla client

def read_config(config_path):
    with open(config_path, 'r') as file:
        cfg = yaml.safe_load(file)
    return cfg

def init_client(host, port):
    try:
        client = carla.Client(host, port)
        print(f"client connected to {host}:{port}")
        print(f"client.get_server_version(): {client.get_server_version()}, client.get_client_version(): {client.get_client_version()}")
        client.set_timeout(60.0)
    except RuntimeError as re:
        if "timeout" not in str(re) and "time-out" not in str(re):
            print("Could not connect to Carla server because:", re)
        client = None 
    return client

# Assuming 'client' is already initialized and is an instance of carla.Client
# The path to your recorder log file

def run_replay(config_path, recorder_file_path, actor_id, start_time, duration):
    cfg = read_config(config_path)
    # server_manager = CarlaServerManager(cfg['carla_sh_path'], cfg['port'], t_sleep=5)
    # server_manager.stop()
    # server_manager.start()

    client = init_client(cfg['host'], cfg['port'])
    # Check if file exists
    try:
        with open(recorder_file_path, 'r') as file:
            print(f"File found: {recorder_file_path}")
    except FileNotFoundError:
        print(f"File not found: {recorder_file_path}")
        exit()
    try:
        # Corrected call to replay_file
        replay_result = client.replay_file(
            recorder_file_path,
            time_start=start_time,  # Explicitly specifying as double
            duration=duration,    # Explicitly specifying as double
            follow_id=actor_id,     # This is already an int, but making sure it's clearly specified
            replay_sensors=True  # Assuming you want to replay sensors, adjust as necessary
        )
        print("Replaying recorded log file...", replay_result)
    except Exception as e:
        print(f"An error occurred while trying to replay the file: {e}")

In [4]:
#Initialize the Carla server

def carla_init():
    global cfg
    global server_manager
    cfg = read_config(config_path)
    server_manager = CarlaServerManager(cfg['carla_sh_path'], cfg['port'], t_sleep=5)
    server_manager.stop()
    server_manager.start()

In [5]:
#Initialize the Carla accident/client

def carla_client():
    global client
    client = init_client(cfg['host'], cfg['port'])

    all_info = client.show_recorder_file_info(recorder_file_path, True)

In [6]:
#Generate all crashes in log file

def crash_gen():
    global collision_info
    collision_info = client.show_recorder_collisions(recorder_file_path, "h", "v")

    print(type(collision_info))
    print(len(collision_info))

    print(collision_info.index("Id"))
    x = collision_info.index("Id")
    print(collision_info[x + 100:x + 104])

    print(collision_info)

    actor_id = int(collision_info[x + 100:x + 104])
    return actor_id

In [7]:
#Gets gaze direction and other data at time of the crash

def crash_info():
    collision_info = client.show_recorder_collisions(recorder_file_path, "h", "v")

    lines = collision_info.splitlines()

    filename = recorder_file_path[: len(recorder_file_path) - 3] + 'csv'
    filename2 = recorder_file_path[: len(recorder_file_path) - 3] + 'json'

    f = open(filename2)
    data = json.load(f)

    collision_vehicles = data['hero']['collisions_vehicle']
    collision_pedestrian = data['hero']['collisions_pedestrian']

    valid_gazes = 0
    time_list = []
    other_actor_list = []

    if (len(lines) > 7):
        for i in range(len(lines) - 8):
            error_count = 0

            j = i + 5
            line = lines[j].strip()
            time = int(line.split()[0])

            if (i == 0):
                time_prior = 0
            else:
                line_prior = lines[j - 1].strip()
                time_prior = int(line_prior.split()[0])

            if (time_prior >= time - 3 and time_prior <= time + 3):
                error_count += 1

            row_num = time + 1
            row_num_prior = time_prior + 1
            counter = 0
            counter_prior = 0

            with open(filename, mode = 'r') as file:
                csv_reader = csv.reader(file)
                header = next(csv_reader)

                for row in csv_reader:
                    counter_prior += 1
                    if counter_prior == row_num_prior:
                        prior_gaze = row[13]
                        
            with open(filename, mode = 'r') as file:
                csv_reader = csv.reader(file)
                header = next(csv_reader)

                for row in csv_reader:
                    counter += 1
                    if counter == row_num:
                        gaze = row[13]

                        if prior_gaze == gaze:
                            error_count += 1
                        break

            if error_count == 2:
                continue
            else:
                print(f"Information about crash {valid_gazes + 1} in this experiment on the Carla simulator:")
                
                print()

                print("Gaze direction at time of crash: {}".format(row[13]))

                intensity = collision_vehicles[valid_gazes].get('intensity')
                print("Intensity: {}".format(intensity))

                print("Time: {}".format(time))
            
                step = collision_vehicles[valid_gazes].get('step')
                print("Step: {}".format(step))

                simulation_time = collision_vehicles[valid_gazes].get('simulation_time')
                print("Simulation Time: {}".format(simulation_time))

                collision_type = collision_vehicles[valid_gazes].get('collision_type')
                print("Collision Type: {}".format(collision_type))

                other_actor_id = collision_vehicles[valid_gazes].get('other_actor_id')
                print("Other Actor ID: {}".format(other_actor_id))

                other_actor_type_id = collision_vehicles[valid_gazes].get('other_actor_type_id')
                print("Other Actor Type ID: {}".format(other_actor_type_id))

                print()

                valid_gazes += 1

                time_list.append(time)
                other_actor_list.append(other_actor_id)
    
    return gaze, intensity, time, time_list, valid_gazes, filename, filename2, step, simulation_time, collision_type, other_actor_id, other_actor_list, other_actor_type_id

In [8]:
#Storing actor_id's of ego vehicles in all recorded crashes and other basic crash information

def store_basic_crash_info():
    file_path_basic_data = project_directory + "crash_info_basic.csv"

    file_exists = os.path.exists(file_path)

    data = []

    for i in range(len(time_list)):
        crash_num = str(i + 1)
        data.append({"Crash # in Log File": crash_num, "Town": town, "Traffic Density": traffic_density, "Driver Condition": driver_condition, "Date": date, "Trial Number": trial_num, "Ego Actor ID": actor_id, "Other Actor ID": other_actor_id})

    # Write to a CSV file
    with open(file_path_basic_data, "a", newline="") as csvfile:
        fieldnames = ["Crash # in Log File", "Town", "Traffic Density", "Driver Condition", "Date", "Trial Number", "Ego Actor ID", "Other Actor ID"]  # Define column headers
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        if not file_exists:
            writer.writeheader()

        writer.writerows(data)

In [9]:
#CSV generation

def csv_generation():
    csv_path = project_directory + r"outputs/experiment_1/" + date + r"/diagnostics/" + folder

    csv_name = file_path + "_replay.csv"

    full_csv_path = os.path.join(csv_path, csv_name)

    crash_num = 0

    for i in range(len(time_list)):
        time = time_list[i]
        other_actor_id = other_actor_list[i]

        start = time - 2
        end = time + 2

        first_time = True
        first_time_distance = 0

        oldh_x = 0
        oldh_y = 0
        oldh_z = 0
        oldo_x = 0
        oldo_y = 0
        oldo_z = 0

        velocityh = 0
        velocityo = 0
        oldvelocityh = 0
        oldvelocityo = 0

        accelerationh = 0
        accelerationo = 0

        world = client.get_world()

        while start < end:
            start = start + 0.1
            client.replay_file(
                recorder_file_path,
                time_start = start,  # Explicitly specifying as double
                duration = 0.01,    # Explicitly specifying as double
                follow_id = actor_id,     # This is already an int, but making sure it's clearly specified
                replay_sensors = True  # Assuming you want to replay sensors, adjust as necessary
            )
        
            hero = world.get_actor(actor_id)
            other_vehicle = world.get_actor(other_actor_id)

            print(start)

            print(f"Velocity hero = {velocityh}")
            print(f"Velocity other = {velocityo}")

            print(f"Acceleration hero = {accelerationh}")
            print(f"Acceleration other = {accelerationo}")

            with open(full_csv_path, 'a', newline = '') as file:
                writer = csv.writer(file)

                if first_time_distance == 0:
                    first_time_distance += 1

                    oldh_x = hero.get_location().x
                    oldh_y = hero.get_location().y
                    oldh_z = hero.get_location().z

                    oldo_x = other_vehicle.get_location().x
                    oldo_y = other_vehicle.get_location().y
                    oldo_z = other_vehicle.get_location().z

                    continue
                else:
                    oldvelocityh = velocityh
                    oldvelocityo = velocityo

                    velocityh = ((hero.get_location().x - oldh_x) ** 2 + (hero.get_location().y - oldh_y) ** 2 + (hero.get_location().z - oldh_z) ** 2) ** 0.5/0.1 * 2.23694
                    velocityo = ((other_vehicle.get_location().x - oldo_x) ** 2 + (other_vehicle.get_location().y - oldo_y) ** 2 + (other_vehicle.get_location().z - oldo_z) ** 2) ** 0.5/0.1 * 2.23694
                
                    accelerationh = (velocityh * 0.44704 - oldvelocityh * 0.44704)/0.1
                    accelerationo = (velocityo * 0.44704 - oldvelocityo * 0.44704)/0.1

                    oldh_x = hero.get_location().x
                    oldh_y = hero.get_location().y
                    oldh_z = hero.get_location().z

                    oldo_x = other_vehicle.get_location().x
                    oldo_y = other_vehicle.get_location().y
                    oldo_z = other_vehicle.get_location().z

                if first_time == True:
                    field = ['simulation_time',
                    'ego_vehicle_x_coordinate',
                    'ego_vehicle_y_coordinate',
                    'ego_vehicle_z_coordinate',
                    'ego_vehicle_velocity',
                    'ego_vehicle_acceleration',
                    'other_vehicle_x_coordinate',
                    'other_vehicle_x_coordinate',
                    'other_vehicle_x_coordinate',
                    'other_vehicle_velocity',
                    'other_vehicle_acceleration',
                    ]
        
                    writer.writerow(field)

                    first_time = False

                writer.writerow(
                    [start,
                    hero.get_location().x,
                    hero.get_location().y,
                    hero.get_location().z,
                    velocityh,
                    accelerationh,
                    other_vehicle.get_location().x,
                    other_vehicle.get_location().y,
                    other_vehicle.get_location().z,
                    velocityo,
                    accelerationo
                    ])

        df = pd.read_csv(full_csv_path)

        df.at[crash_num, 'ego_vehicle_velocity'] = 0
        df.at[crash_num, 'ego_vehicle_acceleration'] = 0
        df.at[crash_num + 1, 'ego_vehicle_acceleration'] = 0

        df.at[crash_num, 'other_vehicle_velocity'] = 0
        df.at[crash_num, 'other_vehicle_acceleration'] = 0
        df.at[crash_num + 1, 'other_vehicle_acceleration'] = 0

        crash_num += 41

        df.to_csv(full_csv_path, index = False)

In [10]:
#Image Generation

def process_image(image):
    array = np.frombuffer(image.raw_data, dtype=np.dtype("uint8"))
    array = np.reshape(array, (image.height, image.width, 4))
    array = array[:, :, :3]  # Remove alpha channel
    return array

def save_image(image, output_path):
    image.save_to_disk(output_path)

def image_callback_ego(image):
    global frame_count
    global output_directory_ego
    output_path = os.path.join(output_directory_ego, f"image_{frame_count:06d}.png")
    save_image(image, output_path)
    frame_count += 1

def image_callback_depth_ego(image):
    global frame_count
    global output_directory_depth_ego
    output_path = os.path.join(output_directory_depth_ego, f"image_{frame_count:06d}.png")
    save_image(image, output_path)
    frame_count += 1

def image_callback_other(image):
    global frame_count
    global output_directory_other
    output_path = os.path.join(output_directory_other, f"image_{frame_count:06d}.png")
    save_image(image, output_path)
    frame_count += 1

def image_callback_depth_other(image):
    global frame_count
    global output_directory_depth_other
    output_path = os.path.join(output_directory_depth_other, f"image_{frame_count:06d}.png")
    save_image(image, output_path)
    frame_count += 1

def create_images():
    global frame_count
    global output_directory_ego
    global output_directory_other
    global output_directory_depth_ego
    global output_directory_depth_other

    for i in range(len(time_list)):
        time = time_list[i]
        other_actor_id = other_actor_list[i]

        start_time = time - 3
        duration = time + 3
        frame_count = 0

        client = init_client(cfg['host'], cfg['port'])

        run_replay(config_path, recorder_file_path, actor_id, start_time, duration)

        # Get the world and set no rendering mode and create blueprint library to access rgb camera sensor
        world = client.get_world()

        settings = world.get_settings()
        settings.fixed_delta_seconds = 0.04
        world.apply_settings(settings)

        blueprint_library = world.get_blueprint_library()
        actor = world.get_actor(actor_id)

        # Add a camera sensor to the vehicle
        '''
        blueprint = world.get_blueprint_library().find('sensor.camera.rgb')
        blueprint.set_attribute('image_size_x', '1920')
        blueprint.set_attribute('image_size_y', '1080')
        blueprint.set_attribute('fov', '110')
        blueprint.set_attribute('sensor_tick', '0.5')
        '''
        
        #camera_transform = carla.Transform(carla.Location(x=0, z=1.7))
        #'''
        camera_bp = blueprint_library.find('sensor.camera.rgb')
        camera_transform = carla.Transform(carla.Location(x=0.335, z=1.3875))
        #'''

        #camera = world.spawn_actor(blueprint, camera_transform, attach_to=actor)
        camera = world.spawn_actor(camera_bp, camera_transform, attach_to=actor)
        
        camera.listen(image_callback_ego)

        #output_directory_ego = project_directory + r"crash_images/experiment_1/00-06-31/CogMod-v0_Town04_high_simple/distracted_1_repeat_1_ego_c" + str(i + 1)
        output_directory_ego = dir_path_images + r"/" + file_path + r"_ego_c" + str(i + 1)
        output_directories_ego.append(output_directory_ego)

        if not os.path.exists(output_directory_ego):
            os.makedirs(output_directory_ego)
        
        snapshot = world.get_snapshot()
        start_simulation_time = snapshot.timestamp.elapsed_seconds
        end_simulation_time = start_simulation_time + 10

        while True:
            snapshot = world.get_snapshot()
            current_time = snapshot.timestamp.elapsed_seconds

            if current_time < end_simulation_time:
                world.tick()
            else:
                break

        camera.stop()
        camera.destroy()
        actor.destroy()

        #Other vehicle's perspective
        start_time = time - 3
        duration = time + 3
        frame_count = 0

        client = init_client(cfg['host'], cfg['port'])

        run_replay(config_path, recorder_file_path, actor_id, start_time, duration)

        # Get the world and set no rendering mode and create blueprint library to access rgb camera sensor
        world = client.get_world()

        other_actor = world.get_actor(other_actor_id)

        # Add a camera sensor to the vehicle
        camera_bp = blueprint_library.find('sensor.camera.rgb')
        camera_transform = carla.Transform(carla.Location(x=1.6, z=1.7))
        camera = world.spawn_actor(camera_bp, camera_transform, attach_to=other_actor)

        camera.listen(image_callback_other)

        #output_directory_other = project_directory + r"crash_images/experiment_1/00-06-31/CogMod-v0_Town04_high_simple/distracted_1_repeat_1_other_c" + str(i + 1)
        output_directory_other = dir_path_images + r"/" + file_path + r"_other_c" + str(i + 1)
        
        output_directories_other.append(output_directory_other)
        
        if not os.path.exists(output_directory_other):
            os.makedirs(output_directory_other)

        snapshot = world.get_snapshot()
        start_simulation_time = snapshot.timestamp.elapsed_seconds
        end_simulation_time = start_simulation_time + 10

        while True:
            snapshot = world.get_snapshot()
            current_time = snapshot.timestamp.elapsed_seconds

            if current_time < end_simulation_time:
                world.tick()
            else:
                break

        camera.stop()
        camera.destroy()
        other_actor.destroy()

        #Top view perspective, following ego vehicle
        start_time = time - 3
        duration = time + 3
        frame_count = 0

        client = init_client(cfg['host'], cfg['port'])

        run_replay(config_path, recorder_file_path, actor_id, start_time, duration)

        # Get the world and set no rendering mode and create blueprint library to access rgb camera sensor
        world = client.get_world()

        other_actor = world.get_actor(other_actor_id)

        # Add a camera sensor to the vehicle
        camera_bp = blueprint_library.find('sensor.camera.rgb')

        birds_eye_transform = carla.Transform(
            carla.Location(x=0.0, y=0.0, z=20.0),  # Position the camera above the vehicle
            carla.Rotation(pitch=-90, yaw=0, roll=0)  # Point the camera straight down
        )
        
        camera = world.spawn_actor(camera_bp, birds_eye_transform, attach_to=actor)

        camera.listen(image_callback_ego)

        #output_directory_ego = project_directory + r"crash_images/experiment_1/00-06-31/CogMod-v0_Town04_high_simple/distracted_1_repeat_1_top_ego_c" + str(i + 1)
        output_directory_ego = dir_path_images + r"/" + file_path + r"_top_ego_c" + str(i + 1)

        output_directories_ego.append(output_directory_ego)
        
        if not os.path.exists(output_directory_ego):
            os.makedirs(output_directory_ego)

        snapshot = world.get_snapshot()
        start_simulation_time = snapshot.timestamp.elapsed_seconds
        end_simulation_time = start_simulation_time + 10

        while True:
            snapshot = world.get_snapshot()
            current_time = snapshot.timestamp.elapsed_seconds

            if current_time < end_simulation_time:
                world.tick()
            else:
                break

        camera.stop()
        camera.destroy()
        other_actor.destroy()

        #Top view perspective, following other vehicle
        start_time = time - 3
        duration = time + 3
        frame_count = 0

        client = init_client(cfg['host'], cfg['port'])

        run_replay(config_path, recorder_file_path, actor_id, start_time, duration)

        # Get the world and set no rendering mode and create blueprint library to access rgb camera sensor
        world = client.get_world()

        other_actor = world.get_actor(other_actor_id)

        # Add a camera sensor to the vehicle
        camera_bp = blueprint_library.find('sensor.camera.rgb')

        birds_eye_transform = carla.Transform(
            carla.Location(x=0.0, y=0.0, z=20.0),  # Position the camera above the vehicle
            carla.Rotation(pitch=-90, yaw=0, roll=0)  # Point the camera straight down
        )
        
        camera = world.spawn_actor(camera_bp, birds_eye_transform, attach_to=other_actor)

        camera.listen(image_callback_other)

        #output_directory_other = project_directory + r"crash_images/experiment_1/00-06-31/CogMod-v0_Town04_high_simple/distracted_1_repeat_1_top_other_c" + str(i + 1)
        output_directory_other = dir_path_images + r"/" + file_path + r"_top_other_c" + str(i + 1)
        
        output_directories_other.append(output_directory_other)
        
        if not os.path.exists(output_directory_other):
            os.makedirs(output_directory_other)

        snapshot = world.get_snapshot()
        start_simulation_time = snapshot.timestamp.elapsed_seconds
        end_simulation_time = start_simulation_time + 10

        while True:
            snapshot = world.get_snapshot()
            current_time = snapshot.timestamp.elapsed_seconds

            if current_time < end_simulation_time:
                world.tick()
            else:
                break

        camera.stop()
        camera.destroy()
        other_actor.destroy()

        #Top view perspective, following ego vehicle
        start_time = time - 3
        duration = time + 3
        frame_count = 0

        client = init_client(cfg['host'], cfg['port'])

        run_replay(config_path, recorder_file_path, actor_id, start_time, duration)

        # Get the world and set no rendering mode and create blueprint library to access rgb camera sensor
        world = client.get_world()

        other_actor = world.get_actor(other_actor_id)

        # Add a camera sensor to the vehicle
        camera_bp = blueprint_library.find('sensor.camera.rgb')

        rear_camera_transform = carla.Transform(
            carla.Location(x=-10.0, z=3),  # Place the camera 2 meters behind the vehicle
        )
        
        camera = world.spawn_actor(camera_bp, rear_camera_transform, attach_to=actor)

        camera.listen(image_callback_ego)

        #output_directory_ego = project_directory + r"crash_images/experiment_1/00-06-31/CogMod-v0_Town04_high_simple/distracted_1_repeat_1_3rd_person_ego_c" + str(i + 1)
        output_directory_ego = dir_path_images + r"/" + file_path + r"_3rd_person_ego_c" + str(i + 1)

        output_directories_ego.append(output_directory_ego)
        
        if not os.path.exists(output_directory_ego):
            os.makedirs(output_directory_ego)

        snapshot = world.get_snapshot()
        start_simulation_time = snapshot.timestamp.elapsed_seconds
        end_simulation_time = start_simulation_time + 10

        while True:
            snapshot = world.get_snapshot()
            current_time = snapshot.timestamp.elapsed_seconds

            if current_time < end_simulation_time:
                world.tick()
            else:
                break

        camera.stop()
        camera.destroy()
        other_actor.destroy()

        #3rd person view perspective, following ego vehicle
        start_time = time - 3
        duration = time + 3
        frame_count = 0

        client = init_client(cfg['host'], cfg['port'])

        run_replay(config_path, recorder_file_path, actor_id, start_time, duration)

        # Get the world and set no rendering mode and create blueprint library to access rgb camera sensor
        world = client.get_world()

        other_actor = world.get_actor(other_actor_id)

        # Add a camera sensor to the vehicle
        camera_bp = blueprint_library.find('sensor.camera.rgb')

        rear_camera_transform = carla.Transform(
            carla.Location(x=-10.0, z=3),  # Place the camera 2 meters behind the vehicle
        )
        
        camera = world.spawn_actor(camera_bp, rear_camera_transform, attach_to=other_actor)

        camera.listen(image_callback_other)

        #output_directory_other = project_directory + r"crash_images/experiment_1/00-06-31/CogMod-v0_Town04_high_simple/distracted_1_repeat_1_3rd_person_other_c" + str(i + 1)
        output_directory_other = dir_path_images + r"/" + file_path + r"_3rd_person_other_c" + str(i + 1)

        output_directories_other.append(output_directory_other)
        
        if not os.path.exists(output_directory_other):
            os.makedirs(output_directory_other)

        snapshot = world.get_snapshot()
        start_simulation_time = snapshot.timestamp.elapsed_seconds
        end_simulation_time = start_simulation_time + 10

        while True:
            snapshot = world.get_snapshot()
            current_time = snapshot.timestamp.elapsed_seconds

            if current_time < end_simulation_time:
                world.tick()
            else:
                break

        camera.stop()
        camera.destroy()
        other_actor.destroy()

        #Depth camera perspective, following ego vehicle
        start_time = time - 3
        duration = time + 3
        frame_count = 0

        client = init_client(cfg['host'], cfg['port'])

        run_replay(config_path, recorder_file_path, actor_id, start_time, duration)

        world = client.get_world()

        other_actor = world.get_actor(other_actor_id)

        #camera_transform = carla.Transform(carla.Location(x=0, z=1.7))
        #'''
        camera_bp = blueprint_library.find('sensor.camera.depth')
        camera_transform = carla.Transform(carla.Location(x=0.335, z=1.3875))
        #'''

        #camera = world.spawn_actor(blueprint, camera_transform, attach_to=actor)
        camera = world.spawn_actor(camera_bp, camera_transform, attach_to=actor)

        camera.listen(image_callback_depth_ego)

        #output_directory_depth_ego = project_directory + r"crash_images/experiment_1/00-06-31/CogMod-v0_Town04_high_simple/distracted_1_repeat_1_depth_ego_c" + str(i + 1)
        output_directory_depth_ego = dir_path_images + r"/" + file_path + r"_depth_ego_c" + str(i + 1)

        output_directories_depth_ego.append(output_directory_depth_ego)

        if not os.path.exists(output_directory_depth_ego):
            os.makedirs(output_directory_depth_ego)
        
        snapshot = world.get_snapshot()
        start_simulation_time = snapshot.timestamp.elapsed_seconds
        end_simulation_time = start_simulation_time + 10

        while True:
            snapshot = world.get_snapshot()
            current_time = snapshot.timestamp.elapsed_seconds

            if current_time < end_simulation_time:
                world.tick()
            else:
                break

        camera.stop()
        camera.destroy()
        other_actor.destroy()

        #Other vehicle's perspective
        start_time = time - 3
        duration = time + 3
        frame_count = 0

        client = init_client(cfg['host'], cfg['port'])

        run_replay(config_path, recorder_file_path, actor_id, start_time, duration)

        # Get the world and set no rendering mode and create blueprint library to access rgb camera sensor
        world = client.get_world()

        other_actor = world.get_actor(other_actor_id)

        # Add a camera sensor to the vehicle
        camera_bp = blueprint_library.find('sensor.camera.depth')
        camera_transform = carla.Transform(carla.Location(x=1.6, z=1.7))
        camera = world.spawn_actor(camera_bp, camera_transform, attach_to=other_actor)

        camera.listen(image_callback_depth_other)

        #output_directory_depth_other = project_directory + r"crash_images/experiment_1/00-06-31/CogMod-v0_Town04_high_simple/distracted_1_repeat_1_depth_other_c" + str(i + 1)
        output_directory_depth_other = dir_path_images + r"/" + file_path + r"_depth_other_c" + str(i + 1)

        output_directories_depth_other.append(output_directory_depth_other)
        
        if not os.path.exists(output_directory_depth_other):
            os.makedirs(output_directory_depth_other)

        snapshot = world.get_snapshot()
        start_simulation_time = snapshot.timestamp.elapsed_seconds
        end_simulation_time = start_simulation_time + 10

        while True:
            snapshot = world.get_snapshot()
            current_time = snapshot.timestamp.elapsed_seconds

            if current_time < end_simulation_time:
                world.tick()
            else:
                break

        camera.stop()
        camera.destroy()
        other_actor.destroy()
    
    return output_directories_ego, output_directories_other, output_directories_depth_ego, output_directories_depth_other

In [11]:
#Image concatenation

def file_list_ego(folder_paths):
    for i in range(len(folder_paths)):
        curr_dir = folder_paths[i]
        files = os.listdir(curr_dir)
        files.sort()
        ego_images.append([os.path.join(curr_dir, f) for f in files])
        #ego_images.append(os.listdir(curr_dir))
        #ego_images[i].sort()

def file_list_other(folder_paths):
    for i in range(len(folder_paths)):
        curr_dir = folder_paths[i]
        files = os.listdir(curr_dir)
        files.sort()
        other_images.append([os.path.join(curr_dir, f) for f in files])
        #other_images.append(os.listdir(curr_dir))
        #other_images[i].sort()

def file_list_depth_ego(folder_paths):
    for i in range(len(folder_paths)):
        curr_dir = folder_paths[i]
        files = os.listdir(curr_dir)
        files.sort()
        depth_ego_images.append([os.path.join(curr_dir, f) for f in files])
        #other_images.append(os.listdir(curr_dir))
        #other_images[i].sort()

def file_list_depth_other(folder_paths):
    for i in range(len(folder_paths)):
        curr_dir = folder_paths[i]
        files = os.listdir(curr_dir)
        files.sort()
        depth_other_images.append([os.path.join(curr_dir, f) for f in files])
        #other_images.append(os.listdir(curr_dir))
        #other_images[i].sort()

def file_list_exec(folder_paths_ego, folder_paths_other, folder_paths_depth_ego, folder_paths_depth_other):
    file_list_ego(folder_paths_ego)
    file_list_other(folder_paths_other)
    file_list_depth_ego(folder_paths_depth_ego)
    file_list_depth_other(folder_paths_depth_other)

def concat_images(images, output_path):
    concat_image_paths.append(output_path)
    images_opened = [Image.open(image) for image in images]
    widths, heights = zip(*(image.size for image in images_opened))

    total_width = sum(widths)
    total_height = sum(heights)
    
    concat_image = Image.new('RGBA', (total_width, total_height))
    
    x_offset = 0
    for image in images_opened:
        concat_image.paste(image, (x_offset, 0))
        x_offset += image.width

    concat_image.save(output_path)

def concat_all():
    for i in range(int(len(folder_paths_ego)/3)):
        folder_paths_ego_concat.append(dir_path_concat + r"/" + file_path + r"_ego_c" + str(i + 1))
        folder_paths_other_concat.append(dir_path_concat + r"/" + file_path + r"_other_c" + str(i + 1))
        folder_paths_ego_concat.append(dir_path_concat + r"/" + file_path + r"_top_ego_c" + str(i + 1))
        folder_paths_other_concat.append(dir_path_concat + r"/" + file_path + r"_top_other_c" + str(i + 1))
        folder_paths_ego_concat.append(dir_path_concat + r"/" + file_path + r"_3rd_person_ego_c" + str(i + 1))
        folder_paths_other_concat.append(dir_path_concat + r"/" + file_path + r"_3rd_person_other_c" + str(i + 1))

    for i in range(int(len(folder_paths_depth_ego))):
        folder_paths_depth_ego_concat.append(dir_path_concat + r"/" + file_path + r"_depth_ego_c" + str(i + 1))
        folder_paths_depth_other_concat.append(dir_path_concat + r"/" + file_path + r"_depth_other_c" + str(i + 1))

    for i in range(len(folder_paths_ego)):
        output_path = folder_paths_ego[i] + '_concatenated.png'
        concat_images(ego_images[i], output_path)
        shutil.move(output_path, folder_paths_ego_concat[i] + '_concatenated.png')

    for i in range(len(folder_paths_other)):
        output_path = folder_paths_other[i] + '_concatenated.png'
        concat_images(other_images[i], output_path)
        shutil.move(output_path, folder_paths_other_concat[i] + '_concatenated.png')

    for i in range(len(folder_paths_depth_ego)):
        output_path = folder_paths_depth_ego[i] + '_concatenated.png'
        concat_images(depth_ego_images[i], output_path)
        shutil.move(output_path, folder_paths_depth_ego_concat[i] + '_concatenated.png')

    for i in range(len(folder_paths_depth_other)):
        output_path = folder_paths_depth_other[i] + '_concatenated.png'
        concat_images(depth_other_images[i], output_path)
        shutil.move(output_path, folder_paths_depth_other_concat[i] + '_concatenated.png')

In [12]:
#GIF Generation

def make_gif(frame_folder, gif_path):
    images = glob.glob(f"{frame_folder}/*.png")

    images.sort()

    frames = [Image.open(image) for image in images]
    frame_one = frames[0]
    frame_one.save(gif_path, format="GIF", append_images=frames, save_all=True, duration=100, loop=0)

def gif_indiv_exec():    
    for i in range(int(len(folder_paths_ego)/3)):
        gif_path_ego.append(dir_path_gifs + r"/" + file_path + r"_ego_c" + str(i + 1) + r".gif")
        gif_path_other.append(dir_path_gifs + r"/" + file_path + r"_other_c" + str(i + 1) + r".gif")
        gif_path_ego.append(dir_path_gifs + r"/" + file_path + r"_top_ego_c" + str(i + 1) + r".gif")
        gif_path_other.append(dir_path_gifs + r"/" + file_path + r"_top_other_c" + str(i + 1) + r".gif")
        gif_path_ego.append(dir_path_gifs + r"/" + file_path + r"_3rd_person_ego_c" + str(i + 1) + r".gif")
        gif_path_other.append(dir_path_gifs + r"/" + file_path + r"_3rd_person_other_c" + str(i + 1) + r".gif")

    for i in range(len(folder_paths_depth_ego)):
        gif_path_depth_ego.append(dir_path_gifs + r"/" + file_path + r"_depth_ego_c" + str(i + 1) + r".gif")
        gif_path_depth_other.append(dir_path_gifs + r"/" + file_path + r"_depth_other_c" + str(i + 1) + r".gif")

    for i in range(len(folder_paths_ego)):
        make_gif(folder_paths_ego[i], gif_path_ego[i])
        make_gif(folder_paths_other[i], gif_path_other[i])

    for i in range(len(folder_paths_depth_ego)):
        make_gif(folder_paths_depth_ego[i], gif_path_depth_ego[i])
        make_gif(folder_paths_depth_other[i], gif_path_depth_other[i])

def combined_gif():
    gif_paths = []

    for i in range(len(gif_path_ego)):
        gif_paths.append(gif_path_ego[i])
        gif_paths.append(gif_path_other[i])

    gifs = [Image.open(gif_path) for gif_path in gif_paths]

    rows, columns = 3, 2
    canvas_width = 2400
    canvas_height = 1200

    # Get the number of frames to use (based on the longest GIF)
    max_frames = 0
    least_frames = 100

    for i in range(len(folder_paths_ego)):
        if len(os.listdir(folder_paths_ego[i])) > max_frames:
            max_frames = len(os.listdir(folder_paths_ego[i]))
        if len(os.listdir(folder_paths_ego[i])) < least_frames:
            least_frames = len(os.listdir(folder_paths_ego[i]))
        if len(os.listdir(folder_paths_other[i])) > max_frames:
            max_frames = len(os.listdir(folder_paths_other[i]))
        if len(os.listdir(folder_paths_other[i])) < least_frames:
            least_frames = len(os.listdir(folder_paths_other[i]))

    # Create a list to hold the combined frames
    combined_frames = []
    frame_list = []

    for a in range(len(time_list)):
        for i in range(least_frames):
            for k in range(2):
                for m in range(3):
                    if k == 0:
                        images = glob.glob(f"{folder_paths_ego[m]}/*.png")
                        images.sort()
                        frame_list.append(images[i])
                    else:
                        images = glob.glob(f"{folder_paths_other[m]}/*.png")
                        images.sort()
                        frame_list.append(images[i])

            combined_image = Image.new('RGB', (canvas_width, canvas_height))
            combined_image.paste(im = Image.open(frame_list[0 + 6*a]), box = (0, 0))
            combined_image.paste(im = Image.open(frame_list[1 + 6*a]), box = (800, 0))
            combined_image.paste(im = Image.open(frame_list[2 + 6*a]), box = (1600, 0))
            combined_image.paste(im = Image.open(frame_list[3 + 6*a]), box = (0, 600))
            combined_image.paste(im = Image.open(frame_list[4 + 6*a]), box = (800, 600))
            combined_image.paste(im = Image.open(frame_list[5 + 6*a]), box = (1600, 600))

            combined_image.resize((int(canvas_width * 0.3), int(canvas_height * 0.3)), Image.LANCZOS)

            combined_frames.append(combined_image)

            frame_list = []

        frame_one = combined_frames[0]

        frame_one.save(
            dir_path_gifs + r"/" + file_path + r"_combined_c1" + ".gif",
            format="GIF",
            append_images=combined_frames[1:],
            save_all=True,
            duration=120,
            loop=0
        )


In [13]:
def main():
    global config_path

    global town, traffic_density, date, driver_condition, trial_num, folder, file_path
    
    global recorder_file_path

    global actor_id

    global gaze, intensity, time, time_list, valid_gazes, filename, filename2, step, simulation_time, collision_type, other_actor_id, other_actor_list, other_actor_type_id

    global output_directories_ego, output_directories_other, output_directories_depth_ego, output_directories_depth_other

    global folder_paths_ego, folder_paths_other, folder_paths_depth_ego, folder_paths_depth_other

    global folder_paths_ego_concat, folder_paths_other_concat, folder_paths_depth_ego_concat, folder_paths_depth_other_concat

    global ego_images, other_images, depth_ego_images, depth_other_images

    global concat_image_paths

    global gif_path_ego, gif_path_other, gif_path_depth_ego, gif_path_depth_other

    global dir_path_images, dir_path_concat, dir_path_gifs

    config_path = project_directory + r"config/benchmark.yaml"
    #recorder_file_path = project_directory + r"accident_dataset/distracted_driver/Town04/low/episode_1.log"
    #recorder_file_path = project_directory + r"outputs/experiment_1/00-06-31/diagnostics/CogMod-v0_Town04_high_simple/distracted_1_repeat_1.log"
    town, traffic_density, date, driver_condition, trial_num, folder, file_path = user_input()
    recorder_file_path = project_directory + r"outputs/experiment_1/" + date + r"/diagnostics/" + folder + r"/" + file_path + ".log"

    print(config_path)
    print(recorder_file_path)
    
    carla_init()
    
    carla_client()

    actor_id = crash_gen()
    
    gaze, intensity, time, time_list, valid_gazes, filename, filename2, step, simulation_time, collision_type, other_actor_id, other_actor_list, other_actor_type_id = crash_info()

    store_basic_crash_info()

    csv_generation()

    output_directories_ego = []
    output_directories_other = []
    output_directories_depth_ego = []
    output_directories_depth_other = []

    dir_path_images = project_directory + r"crash_images/experiment_1/" + date + r"/" + folder + r"/" + file_path
    os.makedirs(dir_path_images)

    output_directories_ego, output_directories_other, output_directories_depth_ego, output_directories_depth_other = create_images()

    folder_paths_ego = output_directories_ego
    folder_paths_other = output_directories_other
    folder_paths_depth_ego = output_directories_depth_ego
    folder_paths_depth_other = output_directories_depth_other

    folder_paths_ego_concat = []
    folder_paths_other_concat = []
    folder_paths_depth_ego_concat = []
    folder_paths_depth_other_concat = []

    dir_path_concat = project_directory + r"concat_images/experiment_1/" + date + r"/" + folder + r"/" + file_path
    os.makedirs(dir_path_concat)
        
    ego_images = []
    other_images = []
    depth_ego_images = []
    depth_other_images = []

    concat_image_paths = []

    gif_path_ego = []
    gif_path_other = []
    gif_path_depth_ego = []
    gif_path_depth_other = []

    dir_path_gifs = project_directory + r"gifs/experiment_1/" + date + r"/" + folder + r"/" + file_path
    os.makedirs(dir_path_gifs)

    file_list_exec(folder_paths_ego, folder_paths_other, folder_paths_depth_ego, folder_paths_depth_other)

    concat_all()
    
    gif_indiv_exec()

    combined_gif()

if __name__ == "__main__":
    main()
    

Town: Town04, Traffic Density: high, Date: 00-06-31, Driver Condition: distracted, Trial Number: 1, Folder: CogMod-v0_Town04_high_simple, File Path: distracted_1_repeat_1
/home/ubuntu/vihaan-devel/carla/sip-report-gen/carla-roach-0.9.13/config/benchmark.yaml
/home/ubuntu/vihaan-devel/carla/sip-report-gen/carla-roach-0.9.13/outputs/experiment_1/00-06-31/diagnostics/CogMod-v0_Town04_high_simple/distracted_1_repeat_1.log


CarlaUE4-Linux: no process found


client connected to localhost:2000
client.get_server_version(): 0.9.13, client.get_client_version(): 0.9.13
<class 'str'>
288
68
4763
Version: 1
Map: Town04
Date: 04/28/24 01:42:28

    Time  Types     Id Actor 1                                 Id Actor 2                            
     174   v v    4763 vehicle.lincoln.mkz_2017              4929 vehicle.tesla.cybertruck           

Frames: 2375
Duration: 237 seconds

Information about crash 1 in this experiment on the Carla simulator:

Gaze direction at time of crash: GazeDirection.LEFTBLINDSPOT
Intensity: 3197.226972899177
Time: 174
Step: 1727
Simulation Time: 172.70000257343054
Collision Type: 1
Other Actor ID: 4929
Other Actor Type ID: vehicle.tesla.cybertruck

172.1
Velocity hero = 0
Velocity other = 0
Acceleration hero = 0
Acceleration other = 0
172.2
Velocity hero = 0
Velocity other = 0
Acceleration hero = 0
Acceleration other = 0
172.29999999999998
Velocity hero = 1.2057121951181442
Velocity other = 1.2057108413540203
Accelera

In [14]:
#LLM Accident Report Generation Experimentation

'''
import base64
import requests
import os
from openai import OpenAI

client = OpenAI()

api_key = os.environ['OPENAI_API_KEY']

encoded_images = []

def encode_image_1(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

def base64_image_size(base64_string):
    # Calculate the size of the base64 string in bytes
    base64_bytes = base64_string.encode('utf-8')
    byte_size = len(base64_bytes)
    
    # Convert bytes to megabytes (1 MB = 1,048,576 bytes)
    megabyte_size = byte_size / (1024 * 1024)
    
    return megabyte_size

# Path to your image
image_path = "/home/ubuntu/vihaan-devel/carla/sip-report-gen/carla-roach-0.9.13/crash_images/experiment_1/00-14-25/CogMod-v0_Town04_high_simple/distracted_1_repeat_1_other_c2_concatenated.png"
#image_path = "/home/ubuntu/Downloads/test.jpg"

file_stats = os.stat(image_path)
print(f'File Size in MegaBytes is {file_stats.st_size / (1024 * 1024)}')

# Getting the base64 string
base64_image = encode_image_1(image_path)

imgsizeb64 = base64_image_size(base64_image)
print(imgsizeb64)

headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {api_key}"
}

payload = {
  "model": "gpt-4o-mini",
  "messages": [
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "What’s in this image?"
        },
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/png;base64,{base64_image}"
          }
        }
      ]
    }
  ],
  "max_tokens": 300
}

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

print(response.json())


def encode_image(image_path_folder):
    for i in range(len(image_path_folder)):
      with open(image_path_folder[i], "rb") as image_file:
          encoded_images.append(base64.b64encode(image_file.read()).decode('utf-8'))

if __name__ == "__main__":
   encode_image(concat_image_paths)

completion = client.chat.completions.create(
  model = "gpt-4o-mini",
  messages = [
     {
        "role": "user",
        "content": [
           {
              "type": "text",
              "text": "Given the following information please create a DMV style report of the following accident in the Carla simulation. I will be giving you the following information of the crash: Images of the crash from the perspectives of both the ego vehicle and other vehicle (PS, if the other vehicle is a firetruck, I won't be able to provide adequate FOV images), 4 human narrative interpretations of the accident, A JSON file including details of the time of the accident as well as the intensity, etc., 2 CSV files (1 includes the location of both vehicles, their acceleration (m/s^2), and velocities (mph). The other includes the gaze direction of the ego vehicle (where the driver was looking) at different time steps). The files are attached. Here are the human interpreted narratives: 1.) Actor 1 merged smoothly but Actor 2 (a big truck) was driving quickly and wasn’t able to slow down causing a crash. 2.) The hero car is trying to make a double lane change on the highway. After the first lane change, a firetruck hits the hero car from behind. 3.) Actor 2 was distracted and collided into actor one. 4.) Firetruck isn't able to slow down in time to avoid collision with Lincoln. The firetruck ends up backending the Lincoln. Liability is very highly debatable since trucks are known to need a while to slow down since their mass is so great. Make your OWN interpretation of the accident in a DMV style report."
           },
           {
              "type": "image_url",
              "image_url": {
                 "url": encoded_images[0]
              }
           },
           {
              "type": "image_url",
              "image_url": {
                 "url": encoded_images[1]
              }
           },
           {
              "type": "image_url",
              "image_url": {
                 "url": encoded_images[2]
              }
           },
           {
              "type": "image_url",
              "image_url": {
                 "url": encoded_images[3]
              }
           },
        ]
     }
  ]
)
'''

'\nimport base64\nimport requests\nimport os\nfrom openai import OpenAI\n\nclient = OpenAI()\n\napi_key = os.environ[\'OPENAI_API_KEY\']\n\nencoded_images = []\n\ndef encode_image_1(image_path):\n  with open(image_path, "rb") as image_file:\n    return base64.b64encode(image_file.read()).decode(\'utf-8\')\n\ndef base64_image_size(base64_string):\n    # Calculate the size of the base64 string in bytes\n    base64_bytes = base64_string.encode(\'utf-8\')\n    byte_size = len(base64_bytes)\n    \n    # Convert bytes to megabytes (1 MB = 1,048,576 bytes)\n    megabyte_size = byte_size / (1024 * 1024)\n    \n    return megabyte_size\n\n# Path to your image\nimage_path = "/home/ubuntu/vihaan-devel/carla/sip-report-gen/carla-roach-0.9.13/crash_images/experiment_1/00-14-25/CogMod-v0_Town04_high_simple/distracted_1_repeat_1_other_c2_concatenated.png"\n#image_path = "/home/ubuntu/Downloads/test.jpg"\n\nfile_stats = os.stat(image_path)\nprint(f\'File Size in MegaBytes is {file_stats.st_size / (102